# Predicting Popularity: Using Text and Content Analysis to Examine Shared Characteristics of Popular Posts on Twitter

### A CS109 Final Project by Belinda Zeng, Roseanne Feng, Yuqi Hou, and Zahra Mahmood

![caption](https://studentshare.net/content/wp-content/uploads/2015/05/53a0e7d640b31_-_unknown-3-51047042.png)

## Background and Motivation

Twitter (https://twitter.com) is social network, real-time news media service, and micro-blogging service where users can use text, photos, and videos to express moments or ideas in 140-characters or less. These 140-character messages are called "tweets.” According to Twitter’s website, millions of tweets are shared in real time, every day. Registered users can read and post tweets, favorite other people’s tweets, retweet other people’s posts, favorite tweets, and follow other accounts. Unregistered users can read tweets from public accounts. 

In today's day and age of Twitter, popularity is measured in hearts, retweets, follows, and follow-backs. What posts get popular over time? What seems to resonate most with people? Do positive or negative sentiments invite more engagement? In this project, we use Twitter's publically available archive of content to  like to examine some of the shared characteristics of popular posts, including length of post, visual content, positivity, negativity.

## Related Work

Our idea came from a desire to understand how movements such as #BlackLivesMatter and #Ferguson begin on Twitter as well as a general desire to know what makes a post popular. We chose to focus on Tweets on an individual level and to use natural language processing to be able to understand and predict what makes posts popular.

One paper that is related to our work is a paper from Cornell titled, [The effect of wording on message propagation: Topic- and author-controlled natural experiments on Twitter](https://chenhaot.com/pages/wording-for-propagation.html), which compaired pairs of tweets containing the same url and written by the same user but employing different wording to see which version attracted more retweets. Twitter itself has published research on [What fuels a Tweet’s engagement?](https://blog.twitter.com/2014/what-fuels-a-tweets-engagement) Their research found that adding video, links and photos all result in an increase in the number of retweets and even breaking down those results by industry. Inspired by previous research, we sought to include sentiment analysis in our understanding of what made a Tweet popular. 

## Initial Questions

1. How does the distribution of retweets and hearts vary for a post depending on the time of day when tweet is created?
2. How does positive and negative sentiment affect popularity? 
3. What Tweets do we think will become popular?

## Data

This data is publicly available via the Twitter Static API that gets queries based on specific parameters. We limited the data set to look at tweets within a specified period of time. We are storing the data in CSV files for now. To reduce file-sizes, we will try to have multiple CSVs so that we don't load too much data into memory. If data exceeds computer memory, we will consider AWS/SQL database alternatives. 

### Scraping

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import csv

In [4]:
# !pip install tweepy

Set up oauth and a app on Twitter (to getthe consumer key & secret and access token and secret)

In [5]:
# great resource where I got all this 
# http://marcobonzanini.com/2015/03/02/mining-twitter-data-with-python-part-1/

import tweepy
import json
from tweepy import OAuthHandler
 
consumer_key = 'lun6TR6KpaISisFdGnQ5Eo8v5'
consumer_secret = 'hmwEtnfvTfI6CljEKKtIGjahG4NcFQvLBXhOnPyFHmAqNZ9fVV'
access_token = '3004335028-UKSgKFDbaBLNWTzXQFrBRDwVOKo0JR475KYY3LW'
access_secret = 'pA6MeW4NYsv3tL0MRvjI1oBqdUZc0os11gesdNVkeLpX2'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)


Create a random sample that consists of 1% of tweets

In [ ]:
# final, final version 

from tweepy import Stream
from tweepy.streaming import StreamListener

# get retweet status
def try_retweet(status, attribute):
    try:
        if getattr(status, attribute):
            return True
    except AttributeError:
        return None

# function that tries to get attribute from object
def try_get(status, attribute):
    try:
        return getattr(status, attribute).encode('utf-8')
    except AttributeError:
        return None

# open csv file
csvFile = open('smallsample.csv', 'a')

# create csv writer
csvWriter = csv.writer(csvFile)

class MyListener(StreamListener):
    
    def on_status(self, status):
        try:
            # save relevant components of the tweet
            
            # get and sanitize hashtags 
            hashtags = status.entities['hashtags']
            hashtag_list = []
            for el in hashtags:
                hashtag_list.append(el['text'])
            hashtag_count = len(hashtag_list)
            
            # get and sanitize urls
            urls = status.entities['urls']
            url_list = []
            for el in urls:
                url_list.append(el['url'])
            url_count = len(url_list)
            
            # get and sanitize user_mentions
            user_mentions = status.entities['user_mentions']
            mention_list = []
            for el in user_mentions:
                mention_list.append(el['screen_name'])
            mention_count = len(mention_list)
            # save it all as a tweet
            tweet = [status.created_at, status.text.encode('utf-8'), status.place, status.lang, status.coordinates, 
              hashtag_list, url_list, mention_list, 
              hashtag_count, url_count, mention_count, 
              try_get(status, 'possibly_sensitive'),
              status.favorite_count, status.favorited, status.retweet_count, status.retweeted, 
              try_retweet(status,'retweeted_status'), 
              try_get(status.user, 'statuses_count'), 
              try_get(status.user, 'favourites_count'), 
              try_get(status.user, 'followers_count'),
              try_get(status.user, 'description'),
              try_get(status.user, 'location')]
            
            # write to csv
            csvWriter.writerow(tweet)
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
    
    # tell us if there's an error
    def on_error(self, status):
        print(status)
        return True

twitter_stream = Stream(auth, MyListener())
twitter_stream.sample()

In [2]:
tweetdf=pd.read_csv("smallsample.csv", names=["created_at", "text", "place", "lang", "coordinates",
                                       "hashtags", "urls", "user_mentions", 
                                       "hashtag_count", "url_count", "mention_count",
                                       "possibly_sensitive", 
                                       "favorite_count", "favorited", "retweet_count", "retweeted",
                                       "retweeted_status", "user_statuses_count", "user_favorites_count",
                                       "user_follower_count", "user_description", "user_location"])
tweetdf.head(10)

,created_at,text,place,lang,coordinates,hashtags,urls,user_mentions,hashtag_count,url_count,mention_count,possibly_sensitive,favorite_count,favorited,retweet_count,retweeted,retweeted_status,user_statuses_count,user_favorites_count,user_follower_count,user_description,user_location
0,2015-11-24 21:24:41,RT @AcWgst: Reminds me of the fairytail~♡ Matt...,NaN,en,NaN,[],[],[u'AcWgst'],0,0,1,NaN,0,False,0,False,True,NaN,NaN,NaN,"I love music, art, history, current events, WD...",California
1,2015-11-24 21:24:41,ドラゴンスラッシュってスマホゲーらしいのだけれどスクリーンショットだけを見てるとほんとヴァニ...,NaN,ja,NaN,[],[],[],0,0,0,NaN,0,False,0,False,NaN,NaN,NaN,NaN,NaN,県北
2,2015-11-24 21:24:41,やられたでや\n石神さん状態w,NaN,ja,NaN,[],[],[],0,0,0,NaN,0,False,0,False,NaN,NaN,NaN,NaN,味楽一条通り店。時々有明 故に我はエロティカセブン,土田 みさと
3,2015-11-24 21:24:41,#UKVOTY1D #MTVStars One Direction (MARO) Nicki...,NaN,sl,NaN,"[u'UKVOTY1D', u'MTVStars']",[],[],2,0,0,NaN,0,False,0,False,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-11-24 21:24:41,"RT @WhylmSingle: I WILL LOOK FOR YOU, I WILL F...",NaN,en,NaN,[],[],[u'WhylmSingle'],0,0,1,NaN,0,False,0,False,True,NaN,NaN,NaN,"Electronic music producer/vocalist, PR/AR mana...","Miami, FL"
5,2015-11-24 21:24:41,RT @higeorgeshelley: If u guys vote enough ton...,NaN,en,NaN,[],[],[u'higeorgeshelley'],0,0,1,NaN,0,False,0,False,True,NaN,NaN,NaN,I wanna runaway...,my own little world
6,2015-11-24 21:24:41,I Been Sleeping On @gilliedakid He 🔥,NaN,en,NaN,[],[],[u'gilliedakid'],0,0,1,NaN,0,False,0,False,NaN,NaN,NaN,NaN,FMOI @quicktriggap Blessed Basketball Bigman S...,Philly
7,2015-11-24 21:24:41,RT @awwmyloueh: @izaynie93 I'm proud of you!,NaN,en,NaN,[],[],"[u'awwmyloueh', u'izaynie93']",0,0,2,NaN,0,False,0,False,True,NaN,NaN,NaN,sometimes I add too little milk to my coffee a...,zquad ◡̈
8,2015-11-24 21:24:41,RT @CaronPeirson: @CleanDropMobile Let's clink...,NaN,en,NaN,[u'DigiBlogChat'],[],"[u'CaronPeirson', u'CleanDropMobile']",1,0,2,NaN,0,False,0,False,True,NaN,NaN,NaN,CleanDrop a mobile app for #foodies who deman...,USA
9,2015-11-24 21:24:41,RT @JosAntonioNez: Dentro de 1 mes todo el mun...,NaN,es,NaN,[],[],[u'JosAntonioNez'],0,0,1,NaN,0,False,0,False,True,NaN,NaN,NaN,Del 97. Hermana de la Macarena. Auxiliar de En...,NaN


In [3]:
tweetdf.shape

(52766, 22)

## Data wrangling

#### Filter for language

In [4]:
df_filtered = tweetdf[tweetdf['lang'] == 'en']

In [5]:
df_filtered.shape

(21336, 22)

In [6]:
# just found this bug with retweet_count, looking into why this might be the case
df_test = tweetdf[tweetdf['retweet_count'] != 0]

In [7]:
df_test.shape

(0, 22)

#### Filter for timezone

In [ ]:
# don't get timezone location so not sure this is possible

#### Popularity formula

In [8]:
popularity = [retweets + favs for retweets, favs in zip(df_filtered.retweet_count, df_filtered.favorite_count)]

#### Add popularity column

In [9]:
df_filtered['popularity'] = popularity

c:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [10]:
df_filtered.shape

(21336, 23)

In [11]:
df_filtered.head()

,created_at,text,place,lang,coordinates,hashtags,urls,user_mentions,hashtag_count,url_count,mention_count,possibly_sensitive,favorite_count,favorited,retweet_count,retweeted,retweeted_status,user_statuses_count,user_favorites_count,user_follower_count,user_description,user_location,popularity
0,2015-11-24 21:24:41,RT @AcWgst: Reminds me of the fairytail~♡ Matt...,NaN,en,NaN,[],[],[u'AcWgst'],0,0,1,NaN,0,False,0,False,True,NaN,NaN,NaN,"I love music, art, history, current events, WD...",California,0
4,2015-11-24 21:24:41,"RT @WhylmSingle: I WILL LOOK FOR YOU, I WILL F...",NaN,en,NaN,[],[],[u'WhylmSingle'],0,0,1,NaN,0,False,0,False,True,NaN,NaN,NaN,"Electronic music producer/vocalist, PR/AR mana...","Miami, FL",0
5,2015-11-24 21:24:41,RT @higeorgeshelley: If u guys vote enough ton...,NaN,en,NaN,[],[],[u'higeorgeshelley'],0,0,1,NaN,0,False,0,False,True,NaN,NaN,NaN,I wanna runaway...,my own little world,0
6,2015-11-24 21:24:41,I Been Sleeping On @gilliedakid He 🔥,NaN,en,NaN,[],[],[u'gilliedakid'],0,0,1,NaN,0,False,0,False,NaN,NaN,NaN,NaN,FMOI @quicktriggap Blessed Basketball Bigman S...,Philly,0
7,2015-11-24 21:24:41,RT @awwmyloueh: @izaynie93 I'm proud of you!,NaN,en,NaN,[],[],"[u'awwmyloueh', u'izaynie93']",0,0,2,NaN,0,False,0,False,True,NaN,NaN,NaN,sometimes I add too little milk to my coffee a...,zquad ◡̈,0


## Exploratory Analysis

After scraping the tweets from the Twitter API, we can use that data to build a feature list that we use to predict how popular an individual tweet is, measured by a composite score based on the amount of retweets and hearts. We will also use metadata to help us analyze trends in the data, for example if there is a correlation between time of day and retweets.

### Yuqi Todo

#### Top 10 hashtags (ranking hashtags by most popular) 

#### Correlation between length of tweet and popularity 

#### Correlation between presence of image and popularity

#### Correlation between presence of links and popularity

#### Correlation between user mentions and popularity

#### Correlation for number of retweets and hearts


### Zahra Todo

#### Correlation between time of day and tweet popularity



#### The distribution of hearts & retweets over time


#### User's followers correlated with popularity

#### What's popular within the 10 most populous cities

#### What's most popular within each state

#### Trending tweets and trending lists affecting virality 

### Sentiment Analysis

#### Determining positive/negative words



In [3]:
#notes: Unicode in texts (probably emoticons? should we find a way to categorize those?)
#df_filtered['text']

#load dicts into lookup, map words to pos or neg value
lookup = {}
with open('positive.txt', 'r') as f:
    for line in f:
        word = line[:-1]
        lookup[word] = 1
with open('negative.txt', 'r') as f:
    for line in f:
        word = line[:-1]
        lookup[word] = -1


[('limited', -1),
 ('disarray', -1),
 ('obstruction', -1),
 ('interference', -1),
 ('magnetic', 1),
 ('desirable', 1),
 ('encouragingly', 1),
 ('sleek', 1),
 ('reservations', -1),
 ('edgy', -1),
 ('protest', -1),
 ('thirst', -1),
 ('controversial', -1),
 ('mirage', -1),
 ('scheming', -1),
 ('famished', -1),
 ('battleground', -1),
 ('integrity', 1),
 ('inexpert', -1),
 ('aggression', -1),
 ('undisputed', 1),
 ('offenses', -1),
 ('unfeeling', -1),
 ('sagely', 1),
 ('asinininity', -1),
 ('eligible', 1),
 ('inhospitality', -1),
 ('seriously', -1),
 ('feeble', -1),
 ('originality', 1),
 ('superficially', -1),
 ('unbearablely', -1),
 ('agitator', -1),
 ('murky', -1),
 ('tenacity', 1),
 ('regal', 1),
 ('immature', -1),
 ('pride', 1),
 ('worth', 1),
 ('divergent', -1),
 ('sinking', -1),
 ('slow-moving', -1),
 ('deluded', -1),
 ('blistering', -1),
 ('rescue', 1),
 ('co-operation', 1),
 ('compassion', 1),
 ('dynamic', 1),
 ('void', -1),
 ('despondency', -1),
 ('misunderstanding', -1),
 ('traumat

#### Visual content


#### Length of post

#### Controversy

### Prediction

In [19]:
import csv

# another example with Cursor get all tweets with a certain hashtag and a certain time frame within past week 
csvFile = open('tweets.csv', 'a')
#Use csv Writer
csvWriter = csv.writer(csvFile)

for tweet in tweepy.Cursor(api.search,q="#PrayForJapan",count=1,\
                           lang="en",\
                           since_id=2015-11-13).items():
    print tweet.created_at, tweet.text
    csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])
    
    

2015-11-14 05:41:34 RT @_MinaDaFly1D: Say no! Say no of the violence in the world! Go to love more 🙏🏻😱❤️ #PrayForJapan
2015-11-14 05:41:34 RT @GEO7GE: Seriously can't believe this has all happened in a day. #PrayForParis #PrayforBeirut #PrayforJapan https://t.co/CYCM2eViSI
2015-11-14 05:41:34 RT @Friendstagram: All we can do is PRAY 🙏🏼🌏

#PrayForParis 
#PrayForJapan
#PrayForLebanon 
#PrayForBaghdad 
#PrayForMexico https://t.co/K2…
2015-11-14 05:41:34 RT @CIothesPorn: My prayers go out to everyone 🙏🏼🙏🏼
#PrayForParis 
#PrayForJapan 
#PrayForLebanon 
#PrayForBaghdad 
#PrayForMexico https://…
2015-11-14 05:41:34 RT @loukeypookey: So sad that this all happened :(
#PrayForJapan
#PrayForLebanon
#PrayForBaghdad
#PrayForMexico
#PrayForParis https://t.co/…
2015-11-14 05:41:34 RT @justinbieburnt: Justin Bieber and his beliebers praying for Paris and Japan. #PrayForParis #PrayForJapan https://t.co/b2yNFYUSwS
2015-11-14 05:41:33 RT @zaynbaabe: Justin's album will still be there tomorrow but some li

TweepError: {"errors":[{"message":"Rate limit exceeded","code":88}]}